# **Entrenamiento de YoloV9**

## **Instalación de las Librerias**

In [ ]:
!pip install ultralytics
!pip install roboflow

## **Importación de las Librerias**

In [ ]:
import os
content= os.getcwd()

from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image
from roboflow import Roboflow
HOME = "/content"
!mkdir -p {HOME}/datasets

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.1/112.6 GB disk)


## **Descargar el dataset de Roboflow**

In [ ]:
## Descargar DATASET
from roboflow import Roboflow
rf = Roboflow(api_key="LYP9jcRo7cBAEBhJIiA8")
project = rf.workspace("sdpfoka").project("my-first-project-suoa8")
version = project.version(3)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...


## **Entrenar el modelo de Yolo con el dataset**

In [ ]:
%cd {HOME}
!yolo task=detect mode=train model=yolov9s.pt data={dataset.location}/data.yaml epochs=125 imgsz=640 plots=True

## **Verificar descarga**

In [ ]:
##Guardar en una variable la carpeta del entrenamiento
!ls runs/detect/train/weights/
model_path = "runs/detect/train/weights/best.pt"
!ls -l {model_path}

best.pt  last.pt
-rw-r--r-- 1 root root 15239874 Aug 14 04:57 runs/detect/train/weights/best.pt


## **Elegir la imagen a predecir**

In [ ]:
# Realizar predicciones basada en el entrenamiento, imagen completa
import google.colab.files

# Upload an image from your local machine
uploaded = google.colab.files.upload()

# Get the path of the uploaded image
image_name = list(uploaded.keys())[0]
image_path = f"/content/{image_name}"


Saving img028.jpeg to img028.jpeg


## **Realizar la predicción**

In [ ]:
!yolo task=detect mode=predict model={model_path} conf=0.25 source={image_path} save_crop=True

## **Realizar la predicción con todas la imagenes**

In [ ]:
test = "/content/test_images"
!yolo task=detect mode=predict model={model_path} conf=0.25 source={test} save_crop=True

## **Realizar predicción para una sola clase**

In [ ]:
# Ejecuta la predicción en la carpeta de imágenes de prueba, mostrando solo la clase 'Stem' (índice 1)
!yolo task=detect mode=predict model={model_path} conf=0.25 source={image_path} classes=[2] show_labels=False

## **Mostrar imagenes**

In [ ]:
from IPython.display import Image
Image(filename='runs/detect/predict4/img028.jpg', width=600)

In [ ]:
Image(filename='runs/detect/predict3/crops/Stem/img00113.jpg', width=600)

# **Analizar afectación del hongo**

In [ ]:
import google.colab.files
!mkdir crops_frijol
!mkdir crops_frijol/healty_leaves
!mkdir crops_frijol/Stem
!mkdir crops_frijol/rooten_leaf

In [ ]:
uploaded = google.colab.files.upload()

image_name = list(uploaded.keys())[0]
image_path = f"/content/crops_frijol/{image_name}"

In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow # Para mostrar imágenes en Colab

def analizar_afectacion_hoja(ruta_imagen):
    """
    Analiza una imagen recortada de una hoja de frijol para determinar el
    porcentaje de tejido sano, afectado y severamente afectado.

    Args:
        ruta_imagen (str): La ruta al archivo de imagen del recorte.

    Returns:
        dict: Un diccionario con los porcentajes de cada categoría.
    """
    # 1. Cargar la imagen
    img = cv2.imread(ruta_imagen)
    if img is None:
        print(f"Error: No se pudo cargar la imagen en {ruta_imagen}")
        return None

    # 2. Convertir la imagen al espacio de color HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 3. Definir los rangos de color para cada estado en HSV
    # Estos valores pueden necesitar ajustes finos según la iluminación y el tipo de cámara.
    # Formato: (H_min, S_min, V_min), (H_max, S_max, V_max)

    # Verde (Sano)
    # Rango de tonos verdes
    verde_bajo = np.array([35, 40, 40])
    verde_alto = np.array([85, 255, 255])

    # Amarillo (Afectado)
    # Rango de tonos amarillos
    amarillo_bajo = np.array([20, 100, 100])
    amarillo_alto = np.array([30, 255, 255])

    # Marrón/Negro (Quemado/Marchito - Severamente Afectado)
    # Este rango es más complejo y puede incluir marrones oscuros y casi negros.
    marron_bajo = np.array([10, 50, 20])
    marron_alto = np.array([20, 255, 200])

    # Podríamos añadir un segundo rango para los tonos más oscuros/quemados si es necesario.


    # 4. Crear máscaras para cada rango de color
    mascara_sana = cv2.inRange(hsv_img, verde_bajo, verde_alto)
    mascara_afectada = cv2.inRange(hsv_img, amarillo_bajo, amarillo_alto)
    mascara_severa = cv2.inRange(hsv_img, marron_bajo, marron_alto)

    # 5. Calcular el área de cada máscara (contando los píxeles blancos)
    pixeles_sanos = cv2.countNonZero(mascara_sana)
    pixeles_afectados = cv2.countNonZero(mascara_afectada)
    pixeles_severos = cv2.countNonZero(mascara_severa)

    # Área total de la hoja analizada (suma de todos los píxeles clasificados)
    # Se excluyen los píxeles del fondo que no caen en ninguna categoría.
    total_pixeles_hoja = pixeles_sanos + pixeles_afectados + pixeles_severos

    if total_pixeles_hoja == 0:
      print(f"Advertencia: No se detectó tejido de planta en {ruta_imagen}")
      return {'sano': 0, 'afectado': 0, 'severo': 0, 'afectacion_total': 0}


    # 6. Calcular los porcentajes
    porcentaje_sano = (pixeles_sanos / total_pixeles_hoja) * 100
    porcentaje_afectado = (pixeles_afectados / total_pixeles_hoja) * 100
    porcentaje_severo = (pixeles_severos / total_pixeles_hoja) * 100

    # El porcentaje de afectación total es la suma de lo afectado y lo severo.
    porcentaje_afectacion_total = porcentaje_afectado + porcentaje_severo

    # 7. Crear la máscara superpuesta
    mascara_superpuesta = np.zeros_like(img) # Inicializar con negro

    # Asignar colores a las áreas clasificadas
    mascara_superpuesta[mascara_sana > 0] = [0, 255, 0]  # Verde para sano (BGR)
    mascara_superpuesta[mascara_afectada > 0] = [0, 255, 255] # Amarillo para afectado (BGR)
    mascara_superpuesta[mascara_severa > 0] = [0, 0, 128] # Marrón oscuro para severo (BGR)


    # # Opcional: Visualizar las máscaras para depuración
    print("Imagen Original:")
    cv2_imshow(img)
    print("Máscara Sana (Verde):")
    cv2_imshow(mascara_sana)
    print("Máscara Afectada (Amarillo):")
    cv2_imshow(mascara_afectada)
    print("Máscara Severa (Marrón):")
    cv2_imshow(mascara_severa)
    print("Máscara Superpuesta:")
    cv2_imshow(mascara_superpuesta)


    return {
        'sano': porcentaje_sano,
        'afectado': porcentaje_afectado,
        'severo': porcentaje_severo,
        'afectacion_total': porcentaje_afectacion_total
    }

In [ ]:
# --- EJEMPLO DE USO ---

# 1. Asegúrate de que tus imágenes recortadas están organizadas en subcarpetas
#    dentro de "crops_frijol", como:
#    crops_frijol/healty_leaves/imgXXX...
#    crops_frijol/rooten_leaf/imgXXX...
#    crops_frijol/Root/imgXXX...
#    crops_frijol/Stem/imgXXX...

# 2. Procesa todas las imágenes en las subcarpetas
ruta_carpeta_crops = 'crops_frijol'
resultados_por_imagen_y_categoria = {} # Diccionario para almacenar resultados agrupados por imagen original y categoría

# Comprueba si la carpeta principal existe
if os.path.exists(ruta_carpeta_crops):
    # Iterar a través de los elementos dentro de la carpeta principal
    for categoria in os.listdir(ruta_carpeta_crops):
        ruta_categoria = os.path.join(ruta_carpeta_crops, categoria)

        # Asegurarse de que es un directorio
        if os.path.isdir(ruta_categoria):
            print(f"\n### Procesando categoría: {categoria} ###")
            lista_imagenes = os.listdir(ruta_categoria)

            for nombre_imagen in lista_imagenes:
                if nombre_imagen.lower().endswith(('.png', '.jpg', '.jpeg')):
                    ruta_completa = os.path.join(ruta_categoria, nombre_imagen)
                    print(f"Analizando: {ruta_completa}")
                    resultado = analizar_afectacion_hoja(ruta_completa)

                    if resultado:
                        # Extraer el número de la imagen original del nombre del archivo
                        # Asumimos que el formato es imgXXX...
                        try:
                            numero_imagen = int(nombre_imagen[3:6]) # Obtiene los 3 dígitos después de 'img'
                        except ValueError:
                            print(f"Advertencia: Nombre de archivo inesperado '{nombre_imagen}'. No se pudo extraer el número de imagen.")
                            continue # Saltar esta imagen si el nombre no coincide con el formato esperado

                        # Agregar el resultado al grupo de la imagen original y categoría
                        if numero_imagen not in resultados_por_imagen_y_categoria:
                            resultados_por_imagen_y_categoria[numero_imagen] = {}

                        if categoria not in resultados_por_imagen_y_categoria[numero_imagen]:
                            resultados_por_imagen_y_categoria[numero_imagen][categoria] = []

                        resultados_por_imagen_y_categoria[numero_imagen][categoria].append(resultado) # Almacenar el diccionario completo de resultados

                        print(f"  -> Resultados para este recorte: Sano={resultado['sano']:.2f}%, Afectado={resultado['afectado']:.2f}%, Severo={resultado['severo']:.2f}%")
                        print("-----------------------------------------")

else:
    print(f"Error: La carpeta '{ruta_carpeta_crops}' no fue encontrada.")
    print("Por favor, crea la carpeta principal y las subcarpetas con tus imágenes recortadas.")


# 3. Calcular y mostrar el promedio de afectación y sano por cada imagen original y categoría
if resultados_por_imagen_y_categoria:
    print(f"\n#################################################")
    print(f"ANÁLISIS RESUMIDO POR IMAGEN ORIGINAL Y CATEGORÍA")
    # Ordenar por número de imagen para una mejor presentación
    for num_imagen in sorted(resultados_por_imagen_y_categoria.keys()):
        print(f"\n--- Imagen {num_imagen:03d} ---")
        for categoria, resultados_recortes in resultados_por_imagen_y_categoria[num_imagen].items():
            if resultados_recortes:
                # Calcular promedios para esta categoría y esta imagen
                promedio_sano = np.mean([res['sano'] for res in resultados_recortes])
                promedio_afectado = np.mean([res['afectado'] for res in resultados_recortes])
                # Incluimos severo en el total afectado para este resumen, si es relevante, o lo mostramos separado
                # Decidimos mostrar Sano y el Afectación Total (Afectado + Severo)
                promedio_afectacion_total = np.mean([res['afectacion_total'] for res in resultados_recortes])


                print(f"  Categoría '{categoria}': Sano = {promedio_sano:.2f}% | Afectación Total = {promedio_afectacion_total:.2f}%")
            else:
                 print(f"  Categoría '{categoria}': No se encontraron recortes válidos para la imagen {num_imagen:03d}.")

    print(f"\n#################################################")
else:
    print("\nNo se analizaron imágenes en las subcarpetas. El cálculo de promedios no es posible.")

# **Descargar proyecto**

In [ ]:
import os
from google.colab import files

# Zip the /content directory
!zip -r /content.zip /content/

# Download the zip file
files.download('/content.zip')

In [ ]:
import zipfile
import os

zip_file_path = '/content/content.zip'
extract_dir = '/'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' unzipped to '{extract_dir}'")